# BERT + LoRA 를 활용하여 Classification 구현하기 (Small Version)

Author : 정상근 (hugmanskj@gmail.com)

> 알림>
> 교육용 목적으로 제작된 코드입니다.

이 노트북에서는 BERT 모델을 LoRa를 이용하여 메모리를 적게 사용하는 방법을 실습합니다.
활용 데이터는 IMDB이며, BERT+LoRA를 이용하여 분류문제를 구현해봅니다.

특히 이번 실습에서는 우선 Small 수준의 BERT를 활용해 LoRA적용 여부만 살펴봅니다.

아래의 소프트웨어와 프레임워크를 사용해서 진행하겠습니다.

(소요시간 : 30분,colab free 기준)

In [ ]:
from IPython.display import Markdown, display

In [ ]:
import os
os.environ["WANDB_MODE"] = "disabled"

In [ ]:
!pip install evaluate==0.4.5

## 향후 쓰일 여러가지 utility function 준비

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params:,} || all params: {all_param:,} || trainable : {100 * trainable_params / all_param}%"
    )
    return trainable_params


def compare_param(ori_p, peft_p):
    """
    Compare two parameter numbers
    """
    print(f"\n# Trainable Parameter \nBefore: {ori_p:>14,d} \nAfter:  {peft_p:>14,d} \nPercentage: {round(peft_p / ori_p * 100, 2)}%")


def show_trainable_structure(model):
    """
    Print only the trainable parameters only with size
    """
    num_totals = 0
    for _, param in model.named_parameters():
        if param.requires_grad:
            print( f"{_} \t {param.shape} \t\t {param.numel():,}" )
            num_totals += 1

    print(f"\nTotal Number of Parameter Names : {num_totals:,}")

## IMDB Dataset

아래 사이트에서 Dataset을 살펴볼 수 있습니다.

https://huggingface.co/datasets/imdb/viewer/plain_text/train

### Dataset 준비

In [ ]:
!pip install -q --upgrade datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.8/494.8 kB 25.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 11.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-nvrtc-cu12 12.5.82 which is incompatible.
torch 2.6

In [ ]:
## It takes time! (about 10min)
from datasets import load_dataset
dataset = load_dataset("imdb")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

In [ ]:
MODEL_ID = "bert-base-uncased"

### Tokenization

tokenized_datasets 에 미리 dataset 전체를 tokenization 해 둡니다.

In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Map:   0%|          | 0/50000 [00:00<?, ? examples/s]

테스트를 위해 작은 데이터 준비

In [ ]:
# -- small dataset (about 40min)
train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(4000))
test_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

# -- full dataset (15 hours)
#train_dataset = tokenized_datasets["train"]
#test_dataset = tokenized_datasets["test"]

## BERT + LoRA 준비

Huggingface 및 PEFT 라이브러리를 이용한 LoRA 셋팅은 크게 아래의 순서로 진행됩니다.

1. Original 모델 준비하기
2. LoRA configuration 준비하기
3. PEFT(LoRA) 화 시킨 Model 준비하기

### Original Model 준비하기

In [ ]:
from transformers import BertForSequenceClassification

model = BertForSequenceClassification.from_pretrained(
    MODEL_ID,
    num_labels=2
)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
ori_p = print_trainable_parameters(model)

trainable params: 109,483,778 || all params: 109,483,778 || trainable : 100.0%


### Model Structure 확인

In [ ]:
show_trainable_structure(model)

bert.embeddings.word_embeddings.weight 	 torch.Size([30522, 768]) 		 23,440,896
bert.embeddings.position_embeddings.weight 	 torch.Size([512, 768]) 		 393,216
bert.embeddings.token_type_embeddings.weight 	 torch.Size([2, 768]) 		 1,536
bert.embeddings.LayerNorm.weight 	 torch.Size([768]) 		 768
bert.embeddings.LayerNorm.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.query.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.query.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.key.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.key.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.self.value.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.self.value.bias 	 torch.Size([768]) 		 768
bert.encoder.layer.0.attention.output.dense.weight 	 torch.Size([768, 768]) 		 589,824
bert.encoder.layer.0.attention.output.dense.bias 	 torch.Size([768]) 		 768
bert.encod

### Original Model 의 성능 확인

In [ ]:
from transformers import TrainingArguments

testing_args = TrainingArguments(
    output_dir='./results',  # 결과 저장 디렉토리
    do_eval=True             # 평가 실행 설정
)

### 평가모듈 준비

In [ ]:
import numpy as np
import evaluate

metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
# Colab 셀에서 바로
%env WANDB_MODE=disabled

env: WANDB_MODE=disabled


In [ ]:
from transformers import Trainer

tester = Trainer(
    model=model,
    args=testing_args,
    eval_dataset=test_dataset,

    # computing metrics (during the training)
    compute_metrics=compute_metrics,
)
res = tester.evaluate()

import pandas as pd
pd.DataFrame([res])

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


eval_loss  eval_model_preparation_time  eval_accuracy  eval_runtime  \
0   0.786165                       0.0052          0.488       29.0185   

   eval_samples_per_second  eval_steps_per_second  
0                   34.461                  4.308

아무 훈련없이 BASE 모델의 성능이 위와 같이 나옴을 확인하였습니다.

### LoRA configuration 준비하기

In [ ]:
from peft import LoraConfig, TaskType

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,  # TASK TYPE
    r=8, # Rank
    lora_alpha=1,
    lora_dropout=0.1,
    #target_modules=["query", "value"],  # We can specify target Modules
)

이 [코드](https://github.com/huggingface/peft/blob/main/src/peft/utils/peft_types.py|)에 다음과 같은 Task Type 을 지원함을 알 수 있습니다.

```
class TaskType(str, enum.Enum):
    SEQ_CLS = "SEQ_CLS"
    SEQ_2_SEQ_LM = "SEQ_2_SEQ_LM"
    CAUSAL_LM = "CAUSAL_LM"
    TOKEN_CLS = "TOKEN_CLS"
    QUESTION_ANS = "QUESTION_ANS"
    FEATURE_EXTRACTION = "FEATURE_EXTRACTION"
```

- 이번 실습에서는, Sequcne Classification 을 수행하기 때문에, SEQ_CLS라는 TaskType을 설정합니다.
- 또한 LoRA 의 alpha 값으로 1.0 정도를 셋팅합니다.
- BERT와 LoRA를 함께 사용하는 예제에서 `target_modules`를 명시하지 않으면 LoRA는 기본적으로 모델 내의 모든 적용 가능한 레이어나 모듈에 자동으로 적용되며, 이 경우 LoRA는 모델 내에서 가장 효과적인 부분, 일반적으로는 attention layer(Query, Key, Value)와 Feed-Forward layer적용됩니다.


### PEFT(LoRA) 화 시킨 Model 준비하기

In [ ]:
from peft import get_peft_model
model = get_peft_model(model, lora_config)

In [ ]:
lora_p = print_trainable_parameters(model)

trainable params: 296,450 || all params: 109,780,228 || trainable : 0.2700395193203643%


In [ ]:
compare_param(ori_p, lora_p)


# Trainable Parameter 
Before:    109,483,778 
After:         296,450 
Percentage: 0.27%


### Fine-Tuning 수행하기

#### Huggingface 의 Trainer 활용하여 훈련하기

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    ## ---- Epoch & Batch ----- ##
    num_train_epochs=8,             # 훈련 에폭 수
    per_device_train_batch_size=16,  # 디바이스 당 훈련 배치 크기
    auto_find_batch_size=True,

    ## ---- Learning Rate ----- ##
    warmup_steps=500,                # 워밍업을 위한 스텝 수
    weight_decay=0.01,               # 가중치 감소율

    ## ---- GPU & NODES ----- ##
    eval_strategy="epoch",

    ## - Logging & Checkpoint - ##
    output_dir='./results',          # 결과와 체크포인트를 저장할 디렉토리


    logging_steps=10,                # 로그를 기록할 스텝 간격
    #save_steps=1000,                 # 체크포인트를 저장할 스텝 간격
)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,

    # on Which Dataset
    train_dataset=train_dataset, # < train
    eval_dataset=test_dataset,   # < test

    # computing metrics (during the training)
    compute_metrics=compute_metrics,
)

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


In [ ]:
import warnings

# huggingface 훈련 중 불필요한 Warning 이 많이 발생함 (epoch 별로)
warnings.filterwarnings("ignore", category=UserWarning)

In [ ]:
trainer.train()

A ConfigError was raised whilst setting the number of model parameters in Weights & Biases config.


KeyboardInterrupt: 

계속해서 성능이 올라가고 있는 것을 확인할 수 있습니다. 즉, Fine-tuning이 이루어지고 있습니다.

#### LoRA BERT 훈련 model structure

In [ ]:
show_trainable_structure(model)